In [25]:
import os
import requests
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import faiss

In [26]:
# Set the Hugging Face authentication token
os.environ["HF_AUTH_TOKEN"] = "Hugging Face API Token"  # Replace with your token
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "true"

# News API key (Replace with your API key from https://newsapi.org/)
news_api_key = "News API Token"

In [27]:
# Function to get news articles related to AI in Healthcare
def get_articles_from_newsapi(query, api_key, page_size=5):
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey={api_key}&pageSize={page_size}"
    response = requests.get(url)
    data = response.json()
    articles = data.get('articles', [])
    if not articles:
        print("No articles found.")
    documents = [article['title'] + ": " + article['description'] for article in articles]
    return documents

In [28]:
# Fetch AI healthcare articles
query = "AI in healthcare"
documents = get_articles_from_newsapi(query, news_api_key)

if not documents:
    print("Error: No documents fetched. Exiting.")
    exit()

In [29]:
documents

['Doctors Say AI Is Introducing Slop Into Patient Care: Early testing demonstrates results that could be disastrous for patients.',
 "VC's healthcare predictions for 2025: more M&A, fierce competition in AI, and a health insurance shake-up under Trump: 13 investors share what they expect to see in healthcare in 2025, from a pickup in healthcare M&A to boosts for Medicare Advantage startups.",
 'Meet 45 rising stars of the venture capital industry who stood out in 2024: Meet the top emerging venture capitalists of 2024, spotlighting key VCs from diverse backgrounds and sectors, including AI, healthcare, and fintech.',
 "Jimini Health is using AI for better mental healthcare. See the 22-slide pitch deck that helped it raise $8 million.: Jimini Health raised $8 million in pre-seed funding to combine AI-assisted mental healthcare with talk therapy. Here's the pitch deck it used.",
 'The Most Dangerous People on the Internet in 2024: From Elon Musk and Donald Trump to state-sponsored hacker

In [30]:
# Initialize models
sentence_model = SentenceTransformer('all-mpnet-base-v2')  # Sentence model for encoding text
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
language_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")

In [31]:
# Create embeddings for documents using the sentence transformer model
embeddings = sentence_model.encode(documents, convert_to_tensor=False)

In [32]:
# Create FAISS index
dimension = embeddings.shape[1]  # Embedding size
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity
index.add(np.array(embeddings).astype(np.float32))  # Add embeddings to FAISS index

In [33]:
# Encode query into embedding
query_embedding = sentence_model.encode([query])

In [34]:
# Perform similarity search in FAISS index
top_k = 3  # Number of relevant documents to retrieve
distances, indices = index.search(np.array(query_embedding).astype(np.float32), top_k)

In [35]:
# Retrieve the most relevant documents based on FAISS search
relevant_docs = [documents[i] for i in indices[0]]

# Combine relevant documents as context for the language model
context = " ".join(relevant_docs)

In [36]:
context

"Doctors Say AI Is Introducing Slop Into Patient Care: Early testing demonstrates results that could be disastrous for patients. Jimini Health is using AI for better mental healthcare. See the 22-slide pitch deck that helped it raise $8 million.: Jimini Health raised $8 million in pre-seed funding to combine AI-assisted mental healthcare with talk therapy. Here's the pitch deck it used. VC's healthcare predictions for 2025: more M&A, fierce competition in AI, and a health insurance shake-up under Trump: 13 investors share what they expect to see in healthcare in 2025, from a pickup in healthcare M&A to boosts for Medicare Advantage startups."

In [37]:
# Define the prompt with the query and context
prompt = f"Generate a comprehensive research paper on the topic: {query}. Focus on the applications, challenges, and future potential of AI in healthcare. Write an introduction, body, and conclusion, based on the following articles: {context}. The content should be detailed and coherent but not exceed 500 words."

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

In [38]:
# Generate a response with the language model
output = language_model.generate(
    **inputs,
    max_length=500,  # Maximum length of generated text
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,  # Control the randomness (lower = more deterministic)
    top_p=0.9,        # Nucleus sampling (top probability)
    top_k=50,         # Top-K sampling (limit top-k candidates)
    no_repeat_ngram_size=2,  # Avoid repeating n-grams
)

In [39]:
# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Generate a comprehensive research paper on the topic: AI in healthcare. Focus on the applications, challenges, and future potential of AI in healthcare. Write an introduction, body, and conclusion, based on the following articles: Doctors Say AI Is Introducing Slop Into Patient Care: Early testing demonstrates results that could be disastrous for patients. Jimini Health is using AI for better mental healthcare. See the 22-slide pitch deck that helped it raise $8 million.: Jimini Health raised $8 million in pre-seed funding to combine AI-assisted mental healthcare with talk therapy. Here's the pitch deck it used. VC's healthcare predictions for 2025: more M&A, fierce competition in AI, and a health insurance shake-up under Trump: 13 investors share what they expect to see in healthcare in 2025, from a pickup in healthcare M&A to boosts for Medicare Advantage startups.. The content should be detailed and coherent but not exceed 500 words. The paper should include a clear thesis statement